In [1]:
import pygame

# --- setup ---
pygame.init()
WIDTH, HEIGHT = 960, 540
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Platformer — Stage 2 with Sky & Ground Textures")
clock = pygame.time.Clock()
font = pygame.font.Font(None, 28)

# --- constants ---
TILE = 48
GRAVITY = 1800
RUN_SPEED = 260
JUMP_SPEED = 720
BG_PARALLAX = 0.35   # how much the sky scrolls vs. the camera (smaller = slower/further away)

# --- load images (change these paths to your files if needed) ---
SKY_PATH = "Graphic/sky.jpg"     # or "/mnt/data/sky.jpg"
GROUND_PATH = "Graphic/ground.png"  # or "/mnt/data/ground.png"

# Sky: opaque background → convert()
sky_img = pygame.image.load(SKY_PATH).convert()
# Ground tile: may have transparency → convert_alpha(); scale to TILE size
ground_tile = pygame.image.load(GROUND_PATH).convert_alpha()
ground_tile = pygame.transform.scale(ground_tile, (TILE, TILE))

# --- helpers ---
def draw_tiled(surface, img, view_rect):
    """
    Draw 'img' repeatedly to cover the screen area represented by view_rect.
    view_rect is the camera window in world coords; we blit relative to it.
    """
    iw, ih = img.get_width(), img.get_height()
    # start tiling from the nearest tile boundary above/left of the view
    start_x = (view_rect.left // iw) * iw
    start_y = (view_rect.top  // ih) * ih
    y = start_y
    while y < view_rect.bottom:
        x = start_x
        while x < view_rect.right:
            surface.blit(img, (x - view_rect.left, y - view_rect.top))
            x += iw
        y += ih

# --- sprites ---
class Block(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        # use the shared ground tile image (copy if you plan to modify per-block)
        self.image = ground_tile
        self.rect = self.image.get_rect(topleft=(x, y))

class Player(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.frames_idle = [self.make_frame((28, 36), body=(235,235,255))]
        self.frames_run  = [self.make_frame((28, 36), leg_shift=i) for i in (0,1,2,3)]
        self.frame_jump  = self.make_frame((28, 36), body=(235,220,220))
        self.frame_index = 0
        self.anim_timer = 0.0
        self.anim_rate = 0.09

        self.image = self.frames_idle[0]
        self.rect = self.image.get_rect(midbottom=(x, y))

        self.pos = pygame.math.Vector2(self.rect.topleft)
        self.vel = pygame.math.Vector2(0, 0)
        self.on_ground = False
        self.facing = 1

    def make_frame(self, size, body=(220,220,240), leg_shift=0):
        surf = pygame.Surface(size, pygame.SRCALPHA).convert_alpha()
        w, h = size
        pygame.draw.rect(surf, body, (4, 2, w-8, h-8), border_radius=6)
        for i in range(2):
            y = h - 10 + ((i + leg_shift) % 2) * 2
            pygame.draw.rect(surf, (180,180,200), (6 + i*10, y, 6, 6), border_radius=2)
        return surf

    def update(self, dt, keys, solids, jump_pressed):
        dx_input = (keys[pygame.K_d] or keys[pygame.K_RIGHT]) - (keys[pygame.K_a] or keys[pygame.K_LEFT])
        self.vel.x = dx_input * RUN_SPEED
        if dx_input: self.facing = 1 if dx_input > 0 else -1

        if jump_pressed and self.on_ground:
            self.vel.y = -JUMP_SPEED
            self.on_ground = False

        self.vel.y += GRAVITY * dt
        if self.vel.y > 2000: self.vel.y = 2000

        # move X then resolve
        self.pos.x += self.vel.x * dt
        self.rect.x = int(self.pos.x)
        for b in pygame.sprite.spritecollide(self, solids, dokill=False):
            if self.vel.x > 0: self.rect.right = b.rect.left
            elif self.vel.x < 0: self.rect.left = b.rect.right
            self.pos.x = self.rect.x

        # move Y then resolve
        self.pos.y += self.vel.y * dt
        self.rect.y = int(self.pos.y)
        landed = False
        for b in pygame.sprite.spritecollide(self, solids, dokill=False):
            if self.vel.y > 0:
                self.rect.bottom = b.rect.top; self.vel.y = 0; landed = True
            elif self.vel.y < 0:
                self.rect.top = b.rect.bottom; self.vel.y = 0
            self.pos.y = self.rect.y
        self.on_ground = landed or (self.on_ground and self.vel.y == 0)

        # animation
        moving = abs(self.vel.x) > 5
        if not self.on_ground:
            base = self.frame_jump
        elif moving:
            self.anim_timer += dt
            if self.anim_timer >= self.anim_rate:
                self.anim_timer -= self.anim_rate
                self.frame_index = (self.frame_index + 1) % len(self.frames_run)
            base = self.frames_run[self.frame_index]
        else:
            base = self.frames_idle[0]
        self.image = pygame.transform.flip(base, self.facing < 0, False)

# --- build level ---
solids = pygame.sprite.Group()
all_sprites = pygame.sprite.Group()

# ground (row of textured tiles)
for gx in range(-5, 40):
    solids.add(Block(gx*TILE, HEIGHT - TILE))

# a few floating platforms
for gx, gy in [(6, 8), (7, 8), (8, 8), (12, 7), (13, 7), (14, 7), (15, 7), (19, 6), (20, 6)]:
    solids.add(Block(gx*TILE, gy*TILE))

player = Player(3*TILE, HEIGHT - TILE)
all_sprites.add(player)

# --- loop ---
running = True
while running:
    dt = clock.tick(60) / 1000.0
    jump_pressed = False

    for event in pygame.event.get():
        if event.type == pygame.QUIT: running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE: running = False
            if event.key == pygame.K_SPACE: jump_pressed = True

    keys = pygame.key.get_pressed()
    player.update(dt, keys, solids, jump_pressed)

    # camera (world → screen offset)
    cam = pygame.math.Vector2(player.rect.centerx - WIDTH//2,
                              player.rect.centery - HEIGHT//2)

    # --- draw ---
    # 1) sky (tiled; parallax so it scrolls slower than the world)
    bg_view = pygame.Rect(int(cam.x * BG_PARALLAX), int(cam.y * BG_PARALLAX), WIDTH, HEIGHT)
    draw_tiled(screen, sky_img, bg_view)

    # 2) solids (textured)
    for block in solids:
        screen.blit(block.image, (block.rect.x - cam.x, block.rect.y - cam.y))

    # 3) player
    screen.blit(player.image, (player.rect.x - cam.x, player.rect.y - cam.y))

    # HUD
    hud = font.render(f"FPS {clock.get_fps():.0f}  on_ground={player.on_ground}", True, (235,235,245))
    screen.blit(hud, (10, 8))

    pygame.display.flip()

pygame.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
